# POP Toy Model

1. Simple POP over 1 time period
2. Multi-Period POP over n time period

## Simple POP Problem

A POP aimed at maximizing profit through promotional strategies for 5 products. 

Discounts, displays, and features has different effects on these products

### Objective

- **Maximize Profit**: Optimize discounts, displays, and features for 5 products to maximize overall profit.

### Variables

- **Discounts** $(x_i)$: Continuous variables representing the discount level for each product $i$, where $i = 1, 2, \ldots, 5$. Each discount variable has specific lower and upper bounds.
- **Display Decisions** $(d_i)$: Binary decisions indicating whether to display each product, with $i = 6, 7, \ldots, 10$.
- **Feature Decisions** $(f_i)$: Binary decisions indicating whether to feature each product, with $i = 11, 12, \ldots, 15$.

### Constraints

1. **Display Constraint**: Exactly one product can be on display.
2. **Feature Constraints**: At least one and no more than two products can be featured.

### Mathematical Formulation

#### Objective Function

The profit function to be maximized is given by:

$$
\text{Maximize} \; \text{Profit} = \sum_{i=1}^{5} 100 \times (1 + (x_i \cdot (1 - x_i) \cdot i)) \times \text{display\_effect}_i \times \text{feature\_effect}_i
$$

where $\text{display\_effect}_i$ and $\text{feature\_effect}_i$ are determined by whether the product is chosen for display or feature, adjusted by unique effect magnitudes for each product.

#### Constraints

1. **Display Constraint**:

$$
\sum_{i=6}^{10} d_i = 1
$$

2. **Feature Constraints**:

- At least one feature:

$$
1 \leq \sum_{i=11}^{15} f_i
$$

- No more than two features:

$$
\sum_{i=11}^{15} f_i \leq 2
$$

This framework defines the optimization problem, where the goal is to find the set of discounts, display decisions, and feature decisions for each product that maximizes total profit while satisfying the constraints on display and feature availability.


In [1]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np

class PromotionOptimizationProblem(ElementwiseProblem):
    def __init__(self):
        self.product_num = 5
        self.discount_num = 1
        self.feature_num = 2
        self.constraint_num = 4
        super().__init__(n_var=self.product_num*(self.discount_num+self.feature_num),  # 5 products * (1 discount + 2 binary decisions for display/feature)
                         n_obj=1,
                         n_constr=self.constraint_num,
                         xl=np.concatenate((np.zeros(5), np.zeros(10))),  # Lower bounds for discount and binary decisions
                         xu=np.concatenate((np.ones(5), np.ones(10))))  # Upper bounds for discount and binary decisions
        self.xl[:5] = 0  # Minimum discount of 0%
        self.xu[:5] = np.array([0.6, 0.5, 0.4, 0.3, 0.2])  # Maximum discounts for each product
        
        # Random magnitudes for each product to generate some soln
        self.discount_effects = np.array([0.5, 1.0, 1.4, 1.05, 1.5])
        self.display_effects = np.array([1.3, 1.6, 1.9, 1.8, 0.2])
        self.feature_effects = np.array([1.6, 1.9, 0.7, 4, 1.05])

    def _evaluate(self, x, out, *args, **kwargs):
        profit = 0
        display_decisions = np.round(x[5:10])
        feature_decisions = np.round(x[10:15])

        for i in range(self.product_num):
            # Quadratic discount effect to generate some soln
            discount_effect = 1 + (x[i] * (1 - x[i]) * i)
            display_effect = self.display_effects[i] if display_decisions[i] > 0 else 1
            feature_effect = self.feature_effects[i] if feature_decisions[i] > 0 else 1
            profit += 100 * discount_effect * display_effect * feature_effect
        
        out["F"] = -profit  # Maximize profit by minimizing negative profit
        
        # Adjusted constraints to ensure exactly 1 display and 1 to 2 features
        out["G"] = [np.sum(x[:5] > 0) - 5,
                    np.sum(x[5:10]) - 1,  # Exactly 1 display per week
                    np.sum(x[10:15]) - 1,  # At least 1 feature per week
                    np.sum(x[10:15]) - 2,]  # No more than 2 features per week

problem = PromotionOptimizationProblem()

algorithm = GA(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 100),
               seed=1,
               verbose=True)



Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.6184654103 |  3.6507062919 |             - |             -
     2 |      200 |  0.5317432392 |  2.2122973784 |             - |             -
     3 |      300 |  0.000000E+00 |  1.4622146506 | -1.161433E+03 | -1.161433E+03
     4 |      400 |  0.000000E+00 |  0.9009600044 | -9.873584E+02 | -1.161433E+03
     5 |      500 |  0.000000E+00 |  0.4938564343 | -9.226364E+02 | -1.161433E+03
     6 |      600 |  0.000000E+00 |  0.2238541779 | -8.451435E+02 | -1.161433E+03
     7 |      700 |  0.000000E+00 |  0.0331602760 | -7.972918E+02 | -1.161433E+03
     8 |      800 |  0.000000E+00 |  0.000000E+00 | -8.660966E+02 | -1.164986E+03
     9 |      900 |  0.000000E+00 |  0.000000E+00 | -1.

In [2]:
# Print the optimal solution found
print("Optimal Solution Found:")
for i in range(5):
    print(f"Product {i+1} - Discount: {res.X[i]:.2f}, Display: {round(res.X[i+5])}, Feature: {round(res.X[i+10])}")

# Calculate and print the total profit from the optimal solution
# Recalculate profit based on the optimal solution to ensure accuracy
profit = 0
for i in range(5):
    discount_effect = 1 + (res.X[i] * (1 - res.X[i]) * problem.discount_effects[i])
    display_effect = problem.display_effects[i] if res.X[i + 5] > 0 else 1
    feature_effect = problem.feature_effects[i] if res.X[i + 10] > 0 else 1
    profit += 100 * discount_effect * display_effect * feature_effect

print(f"Total Profit: {profit:.2f}")

Optimal Solution Found:
Product 1 - Discount: 0.27, Display: 1, Feature: 0
Product 2 - Discount: 0.50, Display: 0, Feature: 0
Product 3 - Discount: 0.40, Display: 0, Feature: 0
Product 4 - Discount: 0.30, Display: 0, Feature: 1
Product 5 - Discount: 0.20, Display: 0, Feature: 0
Total Profit: 1691.14


In [3]:
res.X

array([0.27326287, 0.49999962, 0.4       , 0.3       , 0.2       ,
       0.5207721 , 0.13805484, 0.00544697, 0.1575408 , 0.05318315,
       0.06046036, 0.05172726, 0.09063101, 0.56493373, 0.03626946])

In [4]:
# constraint violation
res.CV

array([0.])

## Multi-Period POP

## Problem Analysis: Promotion Optimization Problem

Similar to previous problem, we aim to maximize the profit from promotional strategies for 5 products over a period of 5 weeks.

### Objective

- **Maximize Total Profit**: Determine the optimal set of discounts, display, and feature decisions for each product across 5 weeks to maximize total profit.

### Variables

- **Discounts** $(x_{i,w})$: Continuous variables for the discount level of product $i$ in week $w$, where $i = 1, \ldots, 5$ and $w = 1, \ldots, 5$.
- **Display Decisions** $(d_{i,w})$: Binary decisions indicating whether to display product $i$ in week $w$.
- **Feature Decisions** $(f_{i,w})$: Binary decisions indicating whether to feature product $i$ in week $w$.

### Constraints

- **Display Constraint**: Exactly one product must be on display each week.

### Mathematical Formulation

#### Objective Function

The objective is to maximize the total profit, which is a function of discounts, display, and feature decisions across all products and weeks:

$$
\text{Maximize} \; \text{Profit} = \sum_{w=1}^{5} \sum_{i=1}^{5} 100 \times \left(1 + x_{i,w} \cdot (1 - x_{i,w}) \cdot \text{discount\_effects}_i\right) \times \text{display\_effect}_i^{d_{i,w}} \times \text{feature\_effect}_i^{f_{i,w}}
$$

where $\text{discount\_effects}_i$, $\text{display\_effect}_i$, and $\text{feature\_effect}_i$ are coefficients that represent the sensitivity of profit to discounts, displays, and features for each product.

#### Constraints

For each week $w$, the display constraint is formulated as:

$$
\sum_{i=1}^{5} d_{i,w} = 1
$$

This constraint ensures that exactly one product is displayed each week.


In [13]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np

class PromotionOptimizationProblem(ElementwiseProblem):
    def __init__(self):
        self.product_num = 5
        self.discount_num = 1
        self.feature_num = 2
        self.constraint_num = 4
        self.week_count = 5

        n_var = self.week_count * self.product_num * (self.discount_num+self.feature_num)
        n_constr = self.week_count * 1 
        super().__init__(n_var=n_var, n_obj=1, n_constr=n_constr, xl=np.zeros(n_var), xu=np.ones(n_var))

        self.discount_xu =[0.6, 0.5, 0.4, 0.3, 0.2]
        for week in range(self.week_count):
            for product in range(5):
                self.xu[week * 15 + product] = np.array(self.discount_xu[product])

        self.discount_effects = np.array([1.2, 1.0, 1.4, 1.05, 1.5]) 
        self.display_effects = np.array([1.3, 1.6, 1.1, 1.8, 1.2]) 
        self.feature_effects = np.array([1.6, 1.2, 1.7, 1.3, 1.05])

    def _evaluate(self, x, out, *args, **kwargs):
        profit = 0
        constraints = []

        for week in range(self.week_count):
            week_profit = 0
            display_count = feature_count = discount_count = 0

            for product in range(5):
                idx = week * 15 + product
                discount = x[idx]
                display = round(x[idx + 5])
                feature = round(x[idx + 10])

                discount_effect = 1 + (x[idx] * (1 - x[idx]) * i)
                display_effect = 1.2 if display else 1
                feature_effect = 1.1 if feature else 1

                week_profit += 100 * discount_effect * display_effect * feature_effect
                
                display_count += display
                feature_count += feature
                discount_count += 1 if discount > 0 else 0

                # constraints += [display_count - 1]

            profit += week_profit
            constraints += [display_count - 1]

        out["F"] = -profit
        out["G"] = constraints

problem = PromotionOptimizationProblem()

algorithm = GA(pop_size=10000)  # Increased population size to find feasible soln

res = minimize(problem,
               algorithm,
               ('n_gen', 2),
               seed=1,
               verbose=True)



n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |    10000 |  0.000000E+00 |  7.6400000000 | -4.286806E+03 | -4.304741E+03
     2 |    20000 |  0.000000E+00 |  5.1962000000 | -4.236207E+03 | -4.392093E+03


In [14]:

if res.X is not None:
    print("Optimal Solution Found:")
    for week in range(5):
        for product in range(5):
            idx = week * 15 + product
            print(f"Week {week+1}, Product {product+1} - Discount: {res.X[idx]:.2f}, Display: {round(res.X[idx + 5])}, Feature: {round(res.X[idx + 10])}")
        print("")
    print(f"Total Profit: {-res.F[0]:.2f}")
else:
    print("No valid solution found. Consider relaxing constraints or further increasing population size/generations.")

Optimal Solution Found:
Week 1, Product 1 - Discount: 0.15, Display: 0, Feature: 1
Week 1, Product 2 - Discount: 0.46, Display: 0, Feature: 0
Week 1, Product 3 - Discount: 0.36, Display: 1, Feature: 1
Week 1, Product 4 - Discount: 0.02, Display: 0, Feature: 1
Week 1, Product 5 - Discount: 0.11, Display: 0, Feature: 0

Week 2, Product 1 - Discount: 0.29, Display: 0, Feature: 0
Week 2, Product 2 - Discount: 0.41, Display: 0, Feature: 0
Week 2, Product 3 - Discount: 0.22, Display: 0, Feature: 1
Week 2, Product 4 - Discount: 0.11, Display: 0, Feature: 1
Week 2, Product 5 - Discount: 0.11, Display: 1, Feature: 0

Week 3, Product 1 - Discount: 0.38, Display: 0, Feature: 1
Week 3, Product 2 - Discount: 0.42, Display: 0, Feature: 0
Week 3, Product 3 - Discount: 0.37, Display: 0, Feature: 0
Week 3, Product 4 - Discount: 0.25, Display: 0, Feature: 1
Week 3, Product 5 - Discount: 0.00, Display: 0, Feature: 1

Week 4, Product 1 - Discount: 0.15, Display: 1, Feature: 0
Week 4, Product 2 - Discount:

In [7]:
res.X

array([0.1452511 , 0.46340904, 0.36383587, 0.01676686, 0.10842159,
       0.35197812, 0.37892624, 0.99210266, 0.24703072, 0.25814303,
       0.7296973 , 0.41559242, 0.86741019, 0.765054  , 0.31366722,
       0.28707361, 0.40683956, 0.22079787, 0.1091865 , 0.10792602,
       0.30284574, 0.26090474, 0.43865841, 0.2557337 , 0.73664886,
       0.35519691, 0.25678341, 0.73515564, 0.55953381, 0.37325821,
       0.37790261, 0.42309821, 0.3697278 , 0.2512851 , 0.00235846,
       0.01487449, 0.30399426, 0.4326704 , 0.47293815, 0.08471529,
       0.92096728, 0.44650761, 0.21817963, 0.62664236, 0.62414961,
       0.14630808, 0.24205449, 0.12878682, 0.26023266, 0.14581412,
       0.57516182, 0.21732945, 0.39167801, 0.13739613, 0.33636798,
       0.03126111, 0.1864381 , 0.16953365, 0.93531927, 0.7833444 ,
       0.53946295, 0.36163719, 0.00246858, 0.12193181, 0.13667709,
       0.31711604, 0.38981488, 0.56407342, 0.28262274, 0.3419116 ,
       0.73819748, 0.97195302, 0.96673762, 0.47025252, 0.53367

In [8]:
res.CV

array([0.])

### Observations

1. As we increase the week count, it is harder to find feasible solutions in the initial population. The `pop_size` has to scale
2. Because of 1., the constraints has been simplied.

